## 前回の反省
* エネルギーがマイナスなのはおかしい。
* サンプリングにおいてうまくいかないこともあるようだ

## 今回の試行
レプリカ交換

### 参考文献
* http://tombo.sub.jp/doc/esj55/MCMC.pdf

In [1]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, math, warnings
from sklearn.feature_selection import SelectKBest, f_regression

In [2]:
#valiable
warnings.resetwarnings()

warnings.simplefilter('error')
num_select = 2

MCMC_step = 50
MCMC_step_each = 5

Max_temper = 100
T = [j for j in range(1, Max_temper+1)]
len_T = len(T)

b = [1/ l for l in T]

In [13]:
X = pd.read_table('training_markers.txt', header=None,  sep ='\t')# 説明変数（こっちの特徴量上位5つ取り出す）


data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
y= data_yields.T.iloc[0]
ori_y = data_yields.T.iloc[0]
print(ori_y)

num_sample = X.shape[0]#サンプル数
num_feature = X.shape[1]#特徴量の数

0     1
1     0
2     0
3     1
4     1
5     0
6     0
7     1
8     0
9     1
10    0
11    0
12    1
13    0
14    0
15    0
16    1
17    1
18    1
19    1
Name: 0, dtype: int64


In [4]:
type(X) #pandas.core.frame.DataFrame
X.shape #(20, 20)

type(y) #pandas.core.series.Series
y.shape #(20,)

type(y[0])

numpy.int64

In [5]:
#特徴量の分散や同じ特徴量が存在するのかを確認

from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0)
sel.fit(X)

# get_supportで保持するデータのみをTrue値、そうでないものはFalse値を返します
print(sum(sel.get_support())) #20、すなわち同じ特徴量は存在していないことがわかった

20


In [6]:
def selection(num_select, X, y):
    # num_select個の特徴量を選択
    #print(f_regression(X,y))
    #print(np.linalg.norm(y))
    selector = SelectKBest(score_func=f_regression, k=num_select) #SelectKBest #f_regression...線形回帰
    selector.fit(X, y)
    mask_each = selector.get_support() #各特徴量を選択したか否かのmaskを取得
    return mask_each

In [7]:
"""
def each_energy(mask, y):
    y_list = y.values.tolist()
    energy_find_dic = {}#bestな特徴量:それを選んだときのエネルギー基準値（０となる）
    for i in range(len(mask)):
        if mask[i]==True:
            energy_find_trait = np.dot(np.array(X[:][i]).T, np.array(y_list))
            energy_find_dic[i]=energy_find_trait
    return sum(energy_find_dic.values())

def each_energy(mask, y):
    y_list = y.values.tolist()
    energy_trait_sum = 0
    for i in range(len(mask)):
        if mask[i]==True:
            energy_trait_sum += np.dot(np.array(X[:][i]).T, np.array(y_list))
    return energy_trait_sum
"""

def each_energy(mask_each): #毎回のmaskとyが引数、合致度合いの計算を行う
    energy_trait_sum = 0
    for i in range(len(mask_each)):
        if mask_each[i]==mask_ori[i]:
            energy_trait_sum += 1
    return energy_trait_sum #もとのyのときのmask(これをmaskとしている)との合致度合い（大きい⇒よく合致している）

In [8]:
"""
argument: s is integer
procedure: convert s from 0 to 1 or 1 to 0
return: changed value
"""
def y_renewal(s: int):
    return 1-s

In [9]:
def Montecarlo(T, num_select, MCMC_step_each, E_first, y):
    E = []
    E.append(E_first)
    '''
    for j in range(MCMC_step_each):
        i = random.randrange(num_sample)
        print(i)
        ori_yi = y[i]
        y[i] = y_renewal(y[i])
        print(y)
        try:
            mask_new = selection(num_select, X, y)
        except Warning:
            print("warning is here.")
            continue
        E_now = selected_X_energy - each_energy(mask_new)
        E_last = E[-1]
        r = math.exp(-(E_now/T-E_last/T)) #温度付き
        R = np.random.uniform(0.0, 1.0)
        if R > r :
            y[i] = ori_yi #元に戻す
        else:
            E.append(E_now)
    '''
    for j in range(MCMC_step_each):
        #print(j)
        ori_yi = y[j]
        #print(y[j])
        y[j] = y_renewal(y[j])
        #print(y)
        mask_new = selection(num_select, X, y)
        #print(mask_new)
        E_now = selected_X_energy - each_energy(mask_new)
        #print(E_now)
        E_last = E[-1]
        r = math.exp(-(E_now/T-E_last/T)) #温度付き
        R = np.random.uniform(0.0, 1.0)
        if R > r :
            y[j] = ori_yi #元に戻す
        else:
            E.append(E_now)
    return E, y #エネルギーのリスト #最後のyだけ(毎回更新しているので)

In [10]:
E_lists = []
Y_lists = []

mask_ori = selection(num_select, X, y)
selected_X_energy = each_energy(mask_ori)

for k in range(len_T):
    E, y = Montecarlo(T[k], num_select, MCMC_step_each, 0, y) #0=each_energy(mask_ori, y)-each_energy(mask_ori, y)
    E_lists.append(E)
    Y_lists.append(y)
    y = data_yields.T.iloc[0] #yの初期化

E_lists_df = pd.DataFrame(index=[], columns=[])

del E_lists[0]#元の条件削除⇒len(E_lists)=MCMC_step_each となるはず

i = random.randrange(len_T-1)
#r = math.exp(-E_lists[i+1][-1]*b[i+1]-(-E_lists[i][-1]*b[i])) #温度付き
r = math.exp((b[i+1]-b[i])* (E_lists[i+1][-1]-E_lists[i][-1]))
rnd = np.random.uniform(0.0, 1.0)
if rnd < r:
    E_lists[i][-1], E_lists[i+1][-1] = E_lists[i+1][-1], E_lists[i][-1]

for j in range(int(MCMC_step/MCMC_step_each)):

    for k in range(len_T):
        E, y = Montecarlo(T[k], num_select, MCMC_step_each, E_lists[k][-1], Y_lists[k])
        E_lists.append(E)
        Y_lists.append(y)

    E_lists_df[j] = E_lists[0:len_T]
    del E_lists[0:len_T]#1つ前のループで作られたリストの削除
    del Y_lists[0:len_T]

    i = random.randrange(len_T-1)
    #r = math.exp(-E_lists[i+1][-1]*b[i+1]-(-E_lists[i][-1]*b[i])) #温度付き
    r = math.exp((b[i+1]-b[i])* (E_lists[i+1][-1]-E_lists[i][-1]))
    rnd = np.random.uniform(0.0, 1.0)
    if rnd < r:
        E_lists[i][-1], E_lists[i+1][-1] = E_lists[i+1][-1], E_lists[i][-1]


#各温度でMCMC_step回動作させたことで、{MCMC_step_each個のエネルギーと最終y}のデータがkセットできた
#最後に知りたいのは、最終ykセットと、エネルギーの動き
E_lists_df

,0,1,2,3,4,5,6,7,8,9
0,"[0, 0, 0, 0]","[0, 2, 0, 0, 2, 2]","[0, 0, 0, 0, 2, 2]","[0, 2, 2, 0]","[2, 2, 2, 2, 2, 0]","[0, 0, 0]","[2, 2, 0, 0]","[2, 2, 2, 0, 2, 0]","[4, 2, 2, 4, 2, 2]","[0, 0, 0, 0]"
1,"[0, 2, 0]","[2, 2, 2, 2, 2, 0]","[2, 2, 2, 2, 2, 0]","[2, 2, 0, 0, 2]","[2, 0, 0, 0]","[2, 0, 0, 0, 2]","[2, 0, 0, 0, 2]","[2, 2, 2, 2, 2, 4]","[0, 2, 0]","[2, 0, 0, 2, 2]"
2,"[0, 0, 2, 0, 2]","[2, 2, 0, 0, 2, 2]","[2, 0, 0, 0, 2]","[0, 0, 2, 2, 2]","[2, 0, 0, 2, 2]","[0, 2, 2, 2, 2]","[4, 4, 4, 2, 2, 2]","[2, 2, 2, 0, 0]","[2, 0, 0, 0, 2, 2]","[0, 2, 2, 2, 2, 0]"
3,"[0, 2, 4, 2, 2]","[0, 2, 2, 4, 2]","[2, 2, 0, 2, 0]","[4, 2, 2, 2, 0, 2]","[2, 2, 2, 2, 2, 0]","[0, 2, 2, 2, 0, 4]","[2, 0, 2, 0, 2]","[2, 0, 0, 2, 2, 2]","[0, 2, 2, 2, 2, 0]","[2, 2, 0, 0, 2, 2]"
4,"[0, 2, 2, 2, 0]","[4, 0, 0, 0, 0, 2]","[2, 2, 2, 2, 2, 4]","[0, 2, 0, 2, 2, 2]","[2, 2, 0, 0]","[0, 2, 2, 2, 2, 2]","[4, 4, 4, 2, 2, 2]","[2, 2, 2, 2, 0, 0]","[2, 0, 0, 0, 2, 2]","[0, 2, 2, 2, 2, 0]"
...,...,...,...,...,...,...,...,...,...,...
95,"[0, 0, 0, 0, 2, 2]","[0, 2, 0, 0, 0]","[2, 0, 0, 0, 0, 2]","[2, 2, 2, 2, 2, 0]","[2, 4, 4, 2, 2, 2]","[2, 4, 4, 2, 2, 2]","[2, 2, 0, 2, 0, 2]","[2, 0, 2, 2, 0, 2]","[2, 2, 2, 2, 2, 0]","[2, 2, 2, 2, 2, 0]"
96,"[0, 2, 2, 2, 2, 0]","[2, 2, 2, 4, 2, 2]","[2, 2, 2, 2, 4, 2]","[2, 0, 0, 0, 2, 2]","[0, 2, 0, 2, 0, 2]","[2, 0, 2, 2, 0, 2]","[2, 4, 4, 2, 2, 2]","[2, 4, 4, 2, 2, 2]","[2, 0, 0, 0, 2, 2]","[0, 2, 0, 0, 2, 2]"
97,"[0, 0, 0, 0, 2, 2]","[0, 2, 2, 0, 2]","[2, 0, 0, 0, 0, 2]","[2, 2, 2, 2, 2, 0]","[2, 4, 4, 2, 2, 2]","[2, 4, 4, 2, 2, 2]","[2, 2, 0, 2, 0, 2]","[2, 0, 2, 2, 0, 2]","[2, 2, 2, 2, 2, 0]","[2, 2, 2, 2, 2, 0]"
98,"[0, 2, 2, 2, 2, 0]","[0, 0, 2, 2, 2, 2]","[0, 2, 2, 2, 4, 2]","[0, 0, 0, 0, 2, 2]","[0, 2, 0, 2, 0, 2]","[0, 0, 2, 2, 0, 2]","[0, 4, 4, 2, 2, 2]","[4, 4, 4, 2, 2, 2]","[0, 0, 0, 0, 2, 2]","[0, 2, 0, 0, 2, 2]"


In [11]:
len(Y_lists)==len_T

True

In [14]:
print(ori_y)

0     1
1     0
2     0
3     1
4     1
5     0
6     0
7     1
8     0
9     1
10    0
11    0
12    1
13    0
14    0
15    0
16    1
17    1
18    1
19    1
Name: 0, dtype: int64


In [15]:
Y_lists_df = pd.DataFrame()
for t in range(len_T):
    Y_lists_df[t+1]=Y_lists[t]
Y_lists_df['ori']=ori_y
Y_lists_df

,1,2,3,4,5,6,7,8,9,10,...,92,93,94,95,96,97,98,99,100,ori
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


## 気になるエラー

```
---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
<ipython-input-15-94d601c5e7dc> in <module>
     35     i = random.randrange(len_T-1)
     36     #r = math.exp(-E_lists[i+1][-1]*b[i+1]-(-E_lists[i][-1]*b[i])) #温度付き
---> 37     r = math.exp((b[i+1]-b[i])* (E_lists[i+1][-1]-E_lists[i][-1]))
     38     rnd = np.random.uniform(0.0, 1.0)
     39     if rnd < r:

IndexError: list index out of range

```

```
y_list = y.values.tolist()
AttributeError: 'list' object has no attribute 'values'
``` 

なぜかyがlistと判断されている

```
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py in f_regression(X, y, center)
    302     # convert to p-value
    303     degrees_of_freedom = y.size - (2 if center else 1)
--> 304     F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
    305     pv = stats.f.sf(F, 1, degrees_of_freedom)
    306     return F, pv

RuntimeWarning: divide by zero encountered in true_divide
```